In [ ]:
from losses.Metrics import metric_dice
from tf.project.segmentation.inference.inference import Pipeline
import matplotlib.pyplot as plt
import pandas as pd 
import json
import csv
import SimpleITK as sitk

In [ ]:
target_size = (128, 128, 256)
target_spacing = (4.0, 4.0, 4.0)
target_direction = (1,0,0,0,1,0,0,0,1)
model_path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/SEGMENTATION/training/20210621-14:00:08/trained_model_20210621-14:00:08.h5'
target_origin = None
from_pp = False

csv_path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/SEGMENTATION/training/20210621-14:00:08/inference/dataset_inference_v2.csv'


df = pd.read_csv(csv_path)
dataset = df[['patient_id', 'study', 'study_id', 'ct_img', 'pet_img', 'mask_img']].to_dict('records')
print("")
pipeline = Pipeline(target_size, target_spacing, target_direction, model_path=model_path, target_origin=None, from_pp=False)
print("Pipeline object initialized")

In [ ]:
import time 

start_time = time.time()
directory = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/SEGMENTATION/training/20210621-14:00:08/inference'
nifti_directory = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/SEGMENTATION/training/20210621-14:00:08/inference/nifti_probs' 
liste_paths_images = []
for count, img_path in enumerate(dataset) : 
    print(count)
    study_uid = img_path['study_id']
    patient_id = img_path['patient_id']
    study = img_path['study']
    pet_img = sitk.ReadImage(img_path['pet_img'], sitk.sitkFloat32)
    pet_spacing = pet_img.GetSpacing()
    pet_size = pet_img.GetSize()
    pet_origin = pet_img.GetOrigin()
    pet_direction = pet_img.GetDirection()
    pet_array = sitk.GetArrayFromImage(pet_img)
    pred_img = pipeline(img_path)
    #save predictions as nifti 
    pred_img.SetSpacing(pet_spacing)
    pred_img.SetDirection(pet_direction)
    pred_img.SetOrigin(pet_origin)
    sitk.WriteImage(pred_img, nifti_directory+'/'+study_uid+'_inference_pred.nii')
    img_path['pred_img'] = nifti_directory+'/'+study_uid+'_inference_pred.nii'

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:

filename = 'prediction_dataset_probs.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow([ "patient_id", "study_id", "study", "ct_img", "pet_img", "mask_img", 'pred_img'])
    for serie in dataset: 
        csv_writer.writerow([serie['patient_id'], serie['study_id'], serie['study'], serie['ct_img'], serie['pet_img'], serie['mask_img'], serie['pred_img']])